# Creating a Simple Agent with Tracing

In [1]:
import dotenv
import os

from openai import OpenAI

dotenv.load_dotenv()

if not os.environ.get("OPENAI_API_KEY"):
    print(
        """Error: OPENAI_API_KEY environment variable not set. Please copy the .env.template file as .env and fill it in.
    
    You can execute these commands in the terminal to get started:
    cp .env.template .env
    code .env
    """
    )

# Test OpenAI Access
print(
    OpenAI()
    .responses.create(
        model=os.environ["OPENAI_DEFAULT_MODEL"], input="Say: We are up and running!"
    )
    .output_text
)

We are up and running!


In [3]:
from agents import Agent, Runner, trace
from openai.types.responses import ResponseTextDeltaEvent

Create a simple Nutrition Assistant Agent

In [8]:
nutrition_agent = Agent(
    name="Nutrition Assistant",
    instructions="""
    あなたは、食事を材料ごとに分解して教えてくれる、頼りになる栄養士です。簡潔な回答をしてくれます。回答には日本語を用います。
    """
)

Let's execute the Agent:

In [9]:
with trace("Simple Nutrition Agent"):
    result = await Runner.run(nutrition_agent, "How healthy are bananas?")

print(result)

RunResult:
- Last agent: Agent(name="Nutrition Assistant", ...)
- Final output (str):
    要点だけお伝えします。
    
    - 主な栄養素: カリウム、食物繊維、ビタミンB6、ビタミンC、天然の糖分
    - 一般的な栄養価（中くらいのバナナ1本約118 g）: カロリー約90 kcal、カリウム約420 mg、食物繊維約3 g、ビタミンB6約0.5 mg、ビタミンC約10 mg
    - 健康効果: 心血管のサポート、腸内環境の改善、エネルギー源、満腹感を得やすい
    - 注意点: 糖質が多めなので、糖質制限中は量を調整。熟度で糖の量とGIが変わる（熟バナナは糖が多く、未熟バナナはでんぷん・抗性デンプンが多くGIはやや低め）。
    - 摂り方のコツ: 朝食の一部やヨーグルト・ナッツと組み合わせてタンパク質と脂質を加えると満腹感が持続します。
    
    必要であれば、1日の目標量に合わせた適切な摂取量も一緒に計算します。
- 2 new item(s)
- 1 raw response(s)
- 0 input guardrail result(s)
- 0 output guardrail result(s)
(See `RunResult` for more details)


Streaming the answer to the screen, token by token

In [10]:
response_stream = Runner.run_streamed(nutrition_agent, "How healthy are bananas?")

async for event in response_stream.stream_events():
    if event.type == "raw_response_event" and isinstance(
        event.data, ResponseTextDeltaEvent
    ):
        print(event.data.delta, end="", flush=True)

要点でお伝えします。1本（中くらい、約118 g）の栄養例です。

- カロリー: 約105 kcal
- 炭水化物: 約27 g（うち糖質約14 g、食物繊維約3 g）
- カリウム: 約420 mg
- ビタミンB6: 約0.5 mg
- ビタミンC: 約10 mg
- その他: マグネシウム、マンガン etc.

良い点
- 心臓と血圧を整えるカリウム源
- すばやいエネルギー源（糖質）
- 食物繊維で腸内環境を整える
- 抗酸化物質とビタミン類が摂取できる

注意点
- 糖質が多めなので、糖質制限や糖尿病の食事管理中は量を調整
- 未熟なものはでんぷんが多く、熟すと糖が増える。満腹感や血糖上昇の感じ方が変わる

摂り方のコツ
- タンパク質源（ヨーグルト、ナッツ、チーズなど）と組み合わせると満足感が持続
- 1日2本程度を目安に、熟度を用途で使い分けると良い

必要なら、あなたの目的に合わせた1日分の摂取量も計算します。

_Good Job!_